In [6]:
import pandas as pd
import os
import re
import spacy
from gensim.models.phrases import Phrases, Phraser
from time import time 
import multiprocessing
from gensim.models import Word2Vec
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix

In [7]:
# !conda install -c conda-forge wordcloud --yes

In [37]:
data = pd.read_csv("/Users/vn0w1vm/Downloads/train_pa.csv")

In [38]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [39]:
data = data[['text','label']]

In [40]:
data.rename(columns ={"text":"review"}, inplace=True)
data.head()

,review,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [41]:
df_clean = data.copy()
from nltk.tokenize import RegexpTokenizer
t = time()


tokenizer = RegexpTokenizer(r'\w+')
df_clean['clean'] = df_clean['review'].astype('str') 
df_clean.dtypes

df_clean["tokens"] = df_clean["clean"].apply(tokenizer.tokenize)
# delete Stop Words

print('Time to tokenize everything: {} mins'.format(round((time() - t) / 60, 2)))
df_clean.head()

Time to tokenize everything: 0.07 mins


,review,label,clean,tokens
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...,"[House, Dem, Aide, We, Didn, t, Even, See, Com..."
1,Ever get the feeling your life circles the rou...,0,Ever get the feeling your life circles the rou...,"[Ever, get, the, feeling, your, life, circles,..."
2,"Why the Truth Might Get You Fired October 29, ...",1,"Why the Truth Might Get You Fired October 29, ...","[Why, the, Truth, Might, Get, You, Fired, Octo..."
3,Videos 15 Civilians Killed In Single US Airstr...,1,Videos 15 Civilians Killed In Single US Airstr...,"[Videos, 15, Civilians, Killed, In, Single, US..."
4,Print \nAn Iranian woman has been sentenced to...,1,Print \nAn Iranian woman has been sentenced to...,"[Print, An, Iranian, woman, has, been, sentenc..."


In [45]:
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [47]:
#First defining the X (input), and the y (output)
y = data['label'].values
X = np.array(df_clean["tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [50]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 33731


In [54]:

def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [55]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,
      '\nshape for test set : ', test_vecs_w2v.shape)

shape for training set :  (16640, 300) 
shape for test set :  (4160, 300)


In [65]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=300))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 128)               38528     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [66]:
history = model.fit(train_vecs_w2v, y_train, epochs=20, batch_size=50,
                   validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 16640 samples, validate on 4160 samples
Epoch 1/20
16640/16640 [==============================] - 1s 37us/step - loss: 0.4536 - accuracy: 0.7974 - val_loss: 0.3257 - val_accuracy: 0.8632
Epoch 2/20
16640/16640 [==============================] - 1s 48us/step - loss: 0.3221 - accuracy: 0.8646 - val_loss: 0.2958 - val_accuracy: 0.8800
Epoch 3/20
16640/16640 [==============================] - 1s 49us/step - loss: 0.2900 - accuracy: 0.8810 - val_loss: 0.2833 - val_accuracy: 0.8875
Epoch 4/20
16640/16640 [==============================] - 1s 34us/step - loss: 0.2698 - accuracy: 0.8907 - val_loss: 0.2701 - val_accuracy: 0.8959
Epoch 5/20
16640/16640 [==============================] - 1s 40us/step - loss: 0.2558 - accuracy: 0.8951 - val_loss: 0.2672 - val_accuracy: 0.8990
Epoch 6/20
16640/16640 [==============================] - 1s 35us/step - loss: 0.2513 - accuracy: 0.9037 - val_loss: 0.2679 - val_accuracy: 0.8988
Epoch 7/20
16640/16640 [==============================] - 1s 34us/ste

In [64]:
# import matplotlib.pyplot as plt
# def plot_history(history):
#     acc = history.history['accuracy']
#     val_acc = history.history['val_accuracy']
#     loss = history.history['loss']
#     val_loss = history.history['val_loss']
#     x = range(1, len(acc) + 1)

#     plt.figure(figsize=(12, 5))
#     plt.subplot(1, 2, 1)
#     plt.plot(x, acc, 'b', label='Training acc')
#     plt.plot(x, val_acc, 'r', label='Validation acc')
#     plt.title('Training and validation accuracy')
#     plt.legend()
#     plt.subplot(1, 2, 2)
#     plt.plot(x, loss, 'b', label='Training loss')
#     plt.plot(x, val_loss, 'r', label='Validation loss')
#     plt.title('Training and validation loss')
#     plt.legend()
# plot_history(history)

In [67]:
## CNN

In [68]:
all_words = [word for tokens in X for word in tokens]
all_sentence_lengths = [len(tokens) for tokens in X]
ALL_VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(ALL_VOCAB)))
print("Max sentence length is %s" % max(all_sentence_lengths))


####################### CHANGE THE PARAMETERS HERE #####################################
EMBEDDING_DIM = 300 # how big is each word vector
MAX_VOCAB_SIZE = 18399# how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 53 # max number of words in a comment to use

15981944 words total, with a vocabulary size of 216124
Max sentence length is 24814


In [72]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
data["review"] = data["review"].astype(str)
tokenizer.fit_on_texts(data["review"].tolist())
training_sequences = tokenizer.texts_to_sequences(X_train.tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = w2v_model[word] if word in w2v_model else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)


######################## TRAIN AND TEST SET #################################
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_sequences = tokenizer.texts_to_sequences(X_test.tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 238051 unique tokens.
(238052, 300)


In [83]:
train_cnn_data.shape, test_cnn_data.shape

((16640, 53), (4160, 53))

In [86]:
from keras.layers import concatenate
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = concatenate([convs[0],convs[1],convs[2]],axis=1)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
        # Original Yoon Kim model
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    # Finally, we feed the output into a Sigmoid layer.
    # The reason why sigmoid is used is because we are trying to achieve a binary classification(1,0) 
    # for each of the 6 labels, and the sigmoid function will squash the output between the bounds of 0 and 1.
    preds = Dense(1,activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [87]:
model = ConvNet(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, False)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 53)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 53, 300)      71415600    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 51, 128)      115328      embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 50, 128)      153728      embedding_2[0][0]                
____________________________________________________________________________________________

In [88]:

history = model.fit(train_cnn_data, y_train, epochs=100, batch_size=50,
                   validation_data=(test_cnn_data,y_test))
loss, accuracy = model.evaluate(train_cnn_data, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_cnn_data, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 16640 samples, validate on 4160 samples
Epoch 1/100
16640/16640 [==============================] - 11s 664us/step - loss: 0.4799 - acc: 0.7544 - val_loss: 0.3390 - val_acc: 0.8375
Epoch 2/100
16640/16640 [==============================] - 11s 647us/step - loss: 0.2908 - acc: 0.8730 - val_loss: 0.2968 - val_acc: 0.8608
Epoch 3/100
16640/16640 [==============================] - 10s 599us/step - loss: 0.2116 - acc: 0.9105 - val_loss: 0.2882 - val_acc: 0.8649
Epoch 4/100
16640/16640 [==============================] - 11s 640us/step - loss: 0.1397 - acc: 0.9448 - val_loss: 0.3027 - val_acc: 0.8673
Epoch 5/100
16640/16640 [==============================] - 11s 669us/step - loss: 0.0906 - acc: 0.9661 - val_loss: 0.3535 - val_acc: 0.8651
Epoch 6/100
16640/16640 [==============================] - 10s 611us/step - loss: 0.0684 - acc: 0.9744 - val_loss: 0.4086 - val_acc: 0.8680
Epoch 7/100
16640/16640 [==============================] - 11s 640us/step - loss: 0.0630 - acc: 0.9755 - val_lo

Epoch 59/100
16640/16640 [==============================] - 10s 575us/step - loss: 0.0107 - acc: 0.9963 - val_loss: 0.6622 - val_acc: 0.8654
Epoch 60/100
16640/16640 [==============================] - 10s 594us/step - loss: 0.0087 - acc: 0.9971 - val_loss: 0.6159 - val_acc: 0.8738
Epoch 61/100
16640/16640 [==============================] - 10s 595us/step - loss: 0.0099 - acc: 0.9968 - val_loss: 0.5982 - val_acc: 0.8702
Epoch 62/100
16640/16640 [==============================] - 10s 622us/step - loss: 0.0100 - acc: 0.9971 - val_loss: 0.6390 - val_acc: 0.8728
Epoch 63/100
16640/16640 [==============================] - 10s 615us/step - loss: 0.0062 - acc: 0.9977 - val_loss: 0.7485 - val_acc: 0.8668
Epoch 64/100
16640/16640 [==============================] - 10s 628us/step - loss: 0.0062 - acc: 0.9980 - val_loss: 0.7519 - val_acc: 0.8762
Epoch 65/100
16640/16640 [==============================] - 10s 627us/step - loss: 0.0090 - acc: 0.9973 - val_loss: 0.6978 - val_acc: 0.8731
Epoch 66/100
